In [2]:
from ECC_utils import ECC, Point, inverseModp
import random
import math
from des import DesKey

# Basic Pollard Rho
We use Pollard Rho for the curve to find a collision s.t. cP + dQ = c'P + d'Q.

In [3]:
# Parameters
p = 20376993552394903
a = 10
b = 1
P = Point(1983, 6761152449250519)
n = 1852453970120513
QA= Point(18586784116581871, 12161036958498472)
QB= Point(18432261261031243, 11140924411855488)

E = ECC(p, a, b)
Q = QA

In [ ]:
# Parameters - Example Input - SEEMS TO WORK FINE EVEN THOUGH GCD=2
p = 16001
a = 10
b = 1
P = Point(1654, 7208)
n = 8026
Q = Point(5000, 1283)

E = ECC(p, a, b)

In [ ]:
# Parameters - Guide to Elliptic Curve Cryptography - WORKS FINE AS GCD==1
p = 229
a = 1
b = 44
P = Point(5, 116)
n = 239
Q = Point(155, 166)

E = ECC(p, a, b)

In [ ]:
# Parameters - On Pollard’s rho method for solving the elliptic curve discrete logarithm problem - WORKS FINE AS GCD=1
p = 659
a = 416
b = 569
P = Point(23, 213)
n = 673
Q = Point(150, 25)

E = ECC(p, a, b)

We're looking for collisions on the orbits of X0 and X_0 under f

In [ ]:
def H(X):
    """
    Function applied to a point to assign it to a partition from which the integers a and b can be found
    Parameters:
    P (Point): Input point
    Returns:
    int: a
    int: b
    """
    # H(X) = j where X is in Sj
    # for 1 <= j <= L (number of sets Si) choose aj, bj from [0, n-1]
    a = random.randint(1, 100)
    b = random.randint(1, 100)
    return a, b

def f_no_part(X, c, d):
    """
    Function applied to a point for Pollard Rho
    Parameters:
    X (Point): Input point
    c (int): 
    d (int): TODO WHAT IS THIS?
    Returns:
    Point: f(X)
    """
    # X = cP + dQ so c += a and d += b
    a, b = H(X)
    
    # res = X + aP + bQ
    aP = E.ECPointMult(a, P)
    c = c + a % n
    bQ = E.ECPointMult(b, Q)
    d = d + b % n
    res1 = E.ECPointAddition(X, aP)
    res2 = E.ECPointAddition(res1, bQ)
    return res2, c, d



In [ ]:
def inc_mod_p(x, p):
    return (x + 1) % p
def double_mod_p(x, p):
    return (2*x) % p

def f(X, c, d):
    """
    Function for transforming an EC Point (X = c_i * P + d_i * Q) to a new, random-looking EC Point (c_i+1 * P + d_i+1 * Q)

    Parameters:
    X (Point): The point
    c (int): c s.t. X = cP + dQ
    d (int): d s.t. X = cP + dQ
    
    Returns:
    Point: The new point, X_i+1 = X_i + P or 2*X_i or X_i + Q
    int: c_i+1, after the transformation is applied
    int: c_i+1, after the transformation is applied
    """
    partition = X.x % 3
    if partition == 0:
        return E.ECPointAddition(X, P), inc_mod_p(c, n), d
    if partition == 1:
        return E.ECPointDoubling(X), double_mod_p(c, n), double_mod_p(d, n)
    if partition == 2:
        return E.ECPointAddition(X, Q), c, inc_mod_p(d, n)

In [ ]:
def pollard_rho_ECC(E, P, Q):
    """
    Function for finding a collision

    Parameters:
    E (ECC): The curve to work on
    P (Point): The base point
    Q (Point): The point s.t. Q = lP

    Returns:
    int, int, int, int: c, d, c_, d_ s.t cP + dQ = c_P + d_Q
    """
    c = random.randint(1, p-2)
    d = random.randint(1, p-2)
    # X = cP + dQ
    X = E.ECPointAddition(E.ECPointMult(c, P), E.ECPointMult(d, Q))
    # X_ = f(X)
    X_, c_, d_ = f(X, c, d)

    while X != X_:
        # X = f(X)
        X, c, d = f(X, c, d)

        # X_ = f(f(X_))
        X_inter, c_inter, d_inter = f(X_, c_, d_)
        X_, c_, d_ = f(X_inter, c_inter, d_inter)
    
    assert E.ECPointAddition( E.ECPointMult(c, P) , E.ECPointMult(d, Q) ) == E.ECPointAddition( E.ECPointMult(c_, P) , E.ECPointMult(d_, Q) ) == X == X_
    print(X, X_)
    return c, d, c_, d_

c, d, c_, d_ = pollard_rho_ECC(E, P, Q)

# TODO Check for case where c = c_, d=d_ ? in which case go back and try again

print(f"c={c}, d={d}, c_={c_}, d_={d_}")

with open("output.txt", "w") as f:
    f.write(f"c={c}, d={d}, c_={c_}, d_={d_}")

for ECDH:
(14570089211650398, 11648322174886797) (14570089211650398, 11648322174886797)
c=1312997136292347, d=1257181070902744, c_=618115033144160, d_=1308082788255376
k=1682779984167835
True


In [ ]:
E.ECPointMult(1682779984167835, P)

In [ ]:
def discrete_log(E, P, Q):
    '''
    Function for finding the discrete log Q = kP from a collision X = c*P + d*Q = X_ = c_*P + d_*Q

    Parameters:
    E (ECC): The curve to work on
    P (Point): The base point
    Q (Point): The point s.t. Q = kP

    Returns:
    int: k s.t. Q = kP
    '''
    # c, d, c_, d_ = pollard_rho_ECC(E, P, Q)
    gcd = math.gcd( (d_ - d), n ) # why do we do this for n, not n-1

    if gcd == 1:
        k = (c-c_) * inverseModp(d_-d, n) % n
        print(f"k={k}")
        return int(k)
    elif gcd > 1:
        k1 = (c-c_)/gcd * inverseModp((d_-d)/gcd, n/gcd) % n
        ks = [int(k1+(i*n/gcd)) for i in range(gcd)]
        print(f"gcd={gcd}, ks={ks}")
        for k in ks:
            if Q == E.ECPointMult(k, P):
                return k
        print(f"Failure")
    else:
        print(f"gcd={gcd}")


k = discrete_log(E, P, Q)

print(E.ECPointMult(k, P) == Q)

In [15]:
def bits_to_bytes(bits):
    """
    Takes a string of 8 bits and returns a single byte
    """
    x = int(bits, 2)
    b = x.to_bytes(8, byteorder='big')
    print(b)
    return b


# bits_to_byte("00010110")
bits_to_bytes("0001011011101100110011000101010000110010011010000011010011110000")

b'\x16\xec\xccT2h4\xf0'


b'\x16\xec\xccT2h4\xf0'

In [21]:
def decrypt(C):
# 1 find the key
    da = discrete_log(E, P, QA)
    # da = 1682779984167835
    key = E.ECPointMult(da, QB).x
    # key = 6714934996831608
# 2 make it 56 bit binary, parity bits, convert to 8 bytes
    key = format(key, 'b')
    key = "0"*(56-len(key)) + key
    key = key[:7] + "0" + key[7:14] + "0" + key[14:21] + "0" + key[21:28] + "0" + key[28:35] + "0" + key[35:42] + "0" + key[42:49] + "0" + key[49:56] + "0"
    key = int(key, 2)
    key = key.to_bytes(8, byteorder='big')
# 3 use des decrypt
    des_key = DesKey(key)
    C_bin = bytes.fromhex(C)
    M = des_key.decrypt(C_bin)
    M = M.decode('utf-8')
    return M

C = "3da46f7b6fa82f53153908bdadcc742ac38e8691e5208aa4bf6be47240c71e75180b9d1030a00810"
decrypt(C)

'Joyeux Noël et Heureuse Année 2022 !\x02\x02'